# 🍎 Macronutrient Balancing Planner - Interactive Analysis

This notebook provides comprehensive analysis and visualization of the RL-based meal planning system.

## Features:
- 📊 **Interactive Training Progress** - Plotly-powered visualizations
- 🏆 **Agent Performance Comparison** - RL vs Baselines
- 🍽️ **AI Meal Planning Demo** - Live meal generation
- 🔍 **Food Database Explorer** - Interactive data analysis
- 🎯 **User Profile Experimentation** - Real-time parameter testing



## 📚 Setup & Imports

In [1]:
# !pip install nbformat>=4.2.0

In [2]:
import sys
try:
    import nbformat
    print(f"Successfully imported nbformat.")
    print(f"nbformat version: {nbformat.__version__}")
    print(f"nbformat path: {nbformat.__file__}")
except ImportError:
    print("Error: nbformat could not be imported.")

print(f"\nPython executable being used by this notebook: {sys.executable}")

Successfully imported nbformat.
nbformat version: 5.10.4
nbformat path: c:\Users\User\OneDrive\Documents\RDS\Y2S3\ML\Macronutrient-Balancing-Planner-using-RL\.venv\Lib\site-packages\nbformat\__init__.py

Python executable being used by this notebook: c:\Users\User\OneDrive\Documents\RDS\Y2S3\ML\Macronutrient-Balancing-Planner-using-RL\.venv\Scripts\python.exe


In [3]:
# Core imports
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.offline as pyo
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Enable plotly in notebook
pyo.init_notebook_mode(connected=True)

# Import our modules
from src.config import Config
from src.data_loader import OpenFoodFactsAPI, RealFoodDatabase
from src.user_profile import UserProfile
from src.evaluation import EvaluationMetrics, BaselineComparison

# Try to import RL components
try:
    from src.environment import MealPlanningEnvironment
    from src.agent import PPOAgent
    RL_AVAILABLE = True
    print("✅ RL components loaded successfully!")
except ImportError as e:
    print("⚠️  RL components not available. Run training first: python main.py --episodes 10")
    print(f"Error: {e}")
    RL_AVAILABLE = False

print("✅ All imports completed!")

✅ RL components loaded successfully!
✅ All imports completed!


## ⚙️ Load Configuration & Components

In [4]:
# Load configuration
config = Config.from_yaml('config.yaml')
config.viz_interactive = True  # Enable interactive plots
print(f"📋 Configuration loaded")

# Setup components
api = OpenFoodFactsAPI(config)
database = RealFoodDatabase(api, config)
if len(database.foods) == 0:
    print("🔄 Populating food database (this may take a moment)...")
    database.populate_database()
else:
    print(f"🍕 Food database loaded with {len(database.foods)} items")

# Import enums for UserProfile
from src.user_profile import Gender, ActivityLevel, Goal

# Create user profile with correct parameters
user_profile = UserProfile(
    age=30, 
    gender=Gender.MALE, 
    weight=75.0, 
    height=180.0,
    activity_level=ActivityLevel.MODERATELY_ACTIVE
)

# Set preferences after creation
user_profile.preferences['liked_categories'] = ['chicken', 'rice', 'vegetables']
user_profile.preferences['disliked_foods'] = ['liver']

# Set weight goal if different from default
user_profile.goals['weight_goal'] = Goal.MAINTAIN_WEIGHT

# Recalculate target calories based on goals
user_profile.goals['target_calories'] = user_profile.calculate_daily_calories()

print(f"👤 User profile: {user_profile.goals['target_calories']} cal/day")


print("✅ All components loaded!")

📋 Configuration loaded
🍕 Food database loaded with 901 items
👤 User profile: 2682 cal/day
✅ All components loaded!


## 🤖 Load Trained Agent

In [5]:
if RL_AVAILABLE:
    # Create environment
    env = MealPlanningEnvironment(database, user_profile, config)
    print(f"🏃‍♂️ Environment created: {env.observation_space.shape} obs, {env.action_space.shape} actions")
    
    # Create and load agent
    obs_dim = env.observation_space.shape[0]
    action_dim = env.action_space.shape[0]
    agent = PPOAgent(obs_dim, action_dim, config)
    
    # Load trained model
    model_path = "models/trained_agent.pth"
    if os.path.exists(model_path):
        agent.load(model_path)
        training_history = agent.training_history
        print(f"🎯 Loaded trained agent with {len(training_history['episode_rewards'])} episodes")
    else:
        print(f"❌ No trained model found. Run: python main.py --episodes 20")
        training_history = None
else:
    print("⚠️  Cannot load agent - RL components not available")
    training_history = None
    env = None
    agent = None

🏃‍♂️ Environment created: (20,) obs, (2,) actions
🎯 Loaded trained agent with 10 episodes


## 📈 Interactive Training Progress

In [6]:
if training_history and len(training_history['episode_rewards']) > 0:
    # Create interactive training progress plot
    rewards = training_history['episode_rewards']
    episodes = list(range(len(rewards)))
    
    # Moving average for smoothing
    window = min(10, len(rewards) // 3)
    if window > 1:
        moving_avg = pd.Series(rewards).rolling(window=window).mean()
    else:
        moving_avg = rewards
    
    fig = go.Figure()
    
    # Raw rewards
    fig.add_trace(go.Scatter(
        x=episodes, y=rewards,
        mode='lines+markers',
        name='Episode Rewards',
        line=dict(color='lightblue', width=1),
        marker=dict(size=4),
        hovertemplate='Episode: %{x}<br>Reward: %{y:.3f}<extra></extra>'
    ))
    
    # Moving average
    if window > 1:
        fig.add_trace(go.Scatter(
            x=episodes, y=moving_avg,
            mode='lines',
            name=f'Moving Average ({window})',
            line=dict(color='red', width=3)
        ))
    
    fig.update_layout(
        title='🚀 Training Progress - Interactive View',
        xaxis_title='Episode',
        yaxis_title='Reward',
        hovermode='x unified',
        height=500,
        template='plotly_white'
    )
    
    fig.show()
    
    # Training statistics
    print("\n" + "="*50)
    print("📊 TRAINING STATISTICS")
    print("="*50)
    print(f"Total Episodes: {len(rewards)}")
    print(f"Final Reward: {rewards[-1]:.3f}")
    print(f"Best Reward: {max(rewards):.3f}")
    print(f"Average Reward: {np.mean(rewards):.3f}")
    print(f"Improvement: {((rewards[-1] - rewards[0]) / abs(rewards[0]) * 100 if rewards[0] != 0 else 0):.1f}%")
else:
    print("❌ No training history available. Run training first.")


📊 TRAINING STATISTICS
Total Episodes: 10
Final Reward: 4.032
Best Reward: 7.134
Average Reward: 4.743
Improvement: 19.4%


## 🏆 Agent Performance Comparison

In [7]:
if RL_AVAILABLE and training_history and agent:
    print("🔄 Running agent evaluation and baseline comparison...")
    
    # Initialize evaluation components
    evaluator = EvaluationMetrics(config)
    baseline_comparison = BaselineComparison(config, database, user_profile)
    
    # Evaluate trained agent
    print("Evaluating RL agent...")
    rl_results = evaluator.evaluate_agent(agent, env, num_episodes=10)
    
    # Run baseline comparisons
    print("Running baseline comparisons...")
    baseline_results = baseline_comparison.evaluate_baselines(env, num_episodes=5)
    
    # Combine results
    all_results = {
        'RL Agent': rl_results,
        **baseline_results
    }
    
    # Create interactive comparison plot
    agents = list(all_results.keys())
    avg_rewards = [result.get_average_reward() for result in all_results.values()]
    success_rates = [result.get_success_rate() for result in all_results.values()]
    
    # Create comparison plot
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Average Reward Comparison', 'Success Rate Comparison'),
        specs=[[{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    # Reward comparison
    colors = ['green' if agent == 'RL Agent' else 'lightcoral' for agent in agents]
    fig.add_trace(
        go.Bar(x=agents, y=avg_rewards, name='Avg Reward', 
               marker_color=colors, text=[f'{r:.2f}' for r in avg_rewards],
               textposition='auto'),
        row=1, col=1
    )
    
    # Success rate comparison
    fig.add_trace(
        go.Bar(x=agents, y=[r*100 for r in success_rates], name='Success Rate (%)',
               marker_color=colors, text=[f'{r:.1%}' for r in success_rates],
               textposition='auto'),
        row=1, col=2
    )
    
    fig.update_layout(
        title='🏆 Agent Performance Comparison',
        height=500,
        showlegend=False,
        template='plotly_white'
    )
    
    fig.show()
    
    # Print detailed results
    print("\n" + "="*60)
    print("🏆 DETAILED COMPARISON RESULTS")
    print("="*60)
    
    for i, (name, result) in enumerate(all_results.items(), 1):
        avg_reward = result.get_average_reward()
        success_rate = result.get_success_rate()
        print(f"{i}. {name:12}: Reward = {avg_reward:8.3f}, Success = {success_rate:.1%}")
        
else:
    print("❌ Cannot run comparison - Agent not available")

🔄 Running agent evaluation and baseline comparison...
Evaluating RL agent...
Running baseline comparisons...



🏆 DETAILED COMPARISON RESULTS
1. RL Agent    : Reward =   23.296, Success = 100.0%
2. Random      : Reward =  -52.769, Success = 0.0%
3. Greedy      : Reward =  -92.231, Success = 0.0%
4. Heuristic   : Reward = -192.735, Success = 0.0%


## 🍽️ Interactive Meal Planning Demo

In [8]:
if RL_AVAILABLE and agent:
    print("🍽️ Generating AI meal plan...")
    
    # Reset environment and generate meal plan
    observation, _ = env.reset()
    meals_data = []
    total_nutrition = {'calories': 0, 'protein': 0, 'carbohydrates': 0, 'fat': 0}
    
    print("\n" + "="*70)
    print("🍽️ AI-GENERATED MEAL PLAN")
    print("="*70)
    
    for step in range(5):  # Generate 5 meals
        # Get action from trained agent
        action, _, _ = agent.select_action(observation, training=False)
        
        # Take step in environment
        observation, reward, done, truncated, info = env.step(action)
        
        if 'selected_food' in info:
            food = info['selected_food']
            portion = info['portion_size']
            meal_nutrition = info['meal_nutrition']
            
            # Store meal data for visualization
            meals_data.append({
                'meal': f'Meal {step + 1}',
                'food': food.name,
                'portion': portion,
                'calories': meal_nutrition['calories'],
                'protein': meal_nutrition['protein'],
                'carbohydrates': meal_nutrition['carbohydrates'],
                'fat': meal_nutrition['fat'],
                'reward': reward
            })
            
            print(f"\n🍴 Meal {step + 1}: {food.name}")
            print(f"   Portion: {portion:.0f}g")
            print(f"   Calories: {meal_nutrition['calories']:.0f} kcal")
            print(f"   P: {meal_nutrition['protein']:.1f}g | C: {meal_nutrition['carbohydrates']:.1f}g | F: {meal_nutrition['fat']:.1f}g")
            print(f"   Reward Score: {reward:.3f}")
            
            # Update totals
            for key in total_nutrition:
                total_nutrition[key] += meal_nutrition[key]
        
        if done or truncated:
            break
    
    # Create interactive meal visualization
    if meals_data:
        df_meals = pd.DataFrame(meals_data)
        
        # Nutritional breakdown chart
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('Calories per Meal', 'Protein Distribution', 
                           'Macronutrient Breakdown', 'Reward Scores'),
            specs=[[{"type": "bar"}, {"type": "bar"}],
                   [{"type": "pie"}, {"type": "bar"}]]
        )
        
        # Calories per meal
        fig.add_trace(
            go.Bar(x=df_meals['meal'], y=df_meals['calories'], 
                   name='Calories', marker_color='orange',
                   text=[f"{cal:.0f}" for cal in df_meals['calories']],
                   textposition='auto'),
            row=1, col=1
        )
        
        # Protein distribution
        fig.add_trace(
            go.Bar(x=df_meals['meal'], y=df_meals['protein'], 
                   name='Protein', marker_color='red',
                   text=[f"{prot:.1f}g" for prot in df_meals['protein']],
                   textposition='auto'),
            row=1, col=2
        )
        
        # Macronutrient pie chart
        fig.add_trace(
            go.Pie(labels=['Protein', 'Carbohydrates', 'Fat'],
                   values=[total_nutrition['protein']*4, 
                          total_nutrition['carbohydrates']*4, 
                          total_nutrition['fat']*9],
                   name='Macros'),
            row=2, col=1
        )
        
        # Reward scores
        fig.add_trace(
            go.Bar(x=df_meals['meal'], y=df_meals['reward'], 
                   name='Reward', marker_color='green',
                   text=[f"{rew:.2f}" for rew in df_meals['reward']],
                   textposition='auto'),
            row=2, col=2
        )
        
        fig.update_layout(
            title='🍽️ Meal Plan Analysis Dashboard',
            height=700,
            showlegend=False,
            template='plotly_white'
        )
        
        fig.show()
    
    # Show targets vs actual
    target_cals = user_profile.goals['target_calories']
    target_protein, target_carbs, target_fat = user_profile.get_target_macro_grams()
    
    print("\n" + "-"*70)
    print("📊 DAILY NUTRITION SUMMARY")
    print("-"*70)
    print(f"Calories: {total_nutrition['calories']:.0f} / {target_cals} ({total_nutrition['calories']/target_cals:.1%})")
    print(f"Protein:  {total_nutrition['protein']:.1f}g / {target_protein:.1f}g ({total_nutrition['protein']/target_protein:.1%})")
    print(f"Carbs:    {total_nutrition['carbohydrates']:.1f}g / {target_carbs:.1f}g ({total_nutrition['carbohydrates']/target_carbs:.1%})")
    print(f"Fat:      {total_nutrition['fat']:.1f}g / {target_fat:.1f}g ({total_nutrition['fat']/target_fat:.1%})")
    
else:
    print("❌ Cannot generate meal plan - Agent not available")

🍽️ Generating AI meal plan...

🍽️ AI-GENERATED MEAL PLAN

🍴 Meal 1: Tartines craquantes quinoa pois chiche
   Portion: 125g
   Calories: 468 kcal
   P: 11.9g | C: 100.1g | F: 1.1g
   Reward Score: 0.434

🍴 Meal 2: Légumes & Quinoa à la provençale
   Portion: 125g
   Calories: 124 kcal
   P: 2.4g | C: 10.9g | F: 7.1g
   Reward Score: 0.558

🍴 Meal 3: Légumes & Quinoa à la provençale
   Portion: 125g
   Calories: 124 kcal
   P: 2.4g | C: 10.9g | F: 7.1g
   Reward Score: 0.516

🍴 Meal 4: Tilda Steamed Brown Basmati and Quinoa
   Portion: 125g
   Calories: 165 kcal
   P: 3.6g | C: 30.1g | F: 2.6g
   Reward Score: 0.700

🍴 Meal 5: Tilda Steamed Brown Basmati and Quinoa
   Portion: 125g
   Calories: 165 kcal
   P: 3.6g | C: 30.1g | F: 2.6g
   Reward Score: 0.783



----------------------------------------------------------------------
📊 DAILY NUTRITION SUMMARY
----------------------------------------------------------------------
Calories: 1047 / 2682 (39.0%)
Protein:  23.9g / 167.6g (14.2%)
Carbs:    182.0g / 301.7g (60.3%)
Fat:      20.6g / 89.4g (23.1%)


## 🔍 Food Database Explorer

In [9]:
# Interactive food database exploration
print(f"🔍 Exploring food database with {len(database.foods)} items...")

# Sample foods for analysis
sample_foods = list(database.foods.values())[:200]  # First 200 foods
foods_data = []

for food in sample_foods:
    foods_data.append({
        'name': food.name[:30],  # Truncate long names
        'calories': food.nutrition.calories,
        'protein': food.nutrition.protein,
        'carbohydrates': food.nutrition.carbohydrates,
        'fat': food.nutrition.fat,
        'quality_score': food.quality_score
    })

df_foods = pd.DataFrame(foods_data)

# Create interactive scatter plot
fig = px.scatter(
    df_foods, 
    x='calories', 
    y='protein',
    size='quality_score',
    color='fat',
    hover_data=['name', 'carbohydrates'],
    title='🍕 Food Database: Calories vs Protein (Size=Quality, Color=Fat)',
    labels={'calories': 'Calories (per 100g)', 'protein': 'Protein (g)'},
    template='plotly_white'
)

fig.update_layout(height=600)
fig.show()

# Food statistics
print("\n" + "="*50)
print("📊 FOOD DATABASE STATISTICS")
print("="*50)
print(df_foods.describe().round(2))

# Top foods by category
print("\n🏆 TOP FOODS BY PROTEIN:")
top_protein = df_foods.nlargest(5, 'protein')[['name', 'protein', 'calories']]
for idx, row in top_protein.iterrows():
    print(f"  {row['name']}: {row['protein']:.1f}g protein, {row['calories']:.0f} kcal")

print("\n🌟 TOP FOODS BY QUALITY:")
top_quality = df_foods.nlargest(5, 'quality_score')[['name', 'quality_score', 'calories']]
for idx, row in top_quality.iterrows():
    print(f"  {row['name']}: Quality {row['quality_score']:.2f}, {row['calories']:.0f} kcal")

🔍 Exploring food database with 901 items...



📊 FOOD DATABASE STATISTICS
       calories  protein  carbohydrates     fat  quality_score
count    200.00   200.00         200.00  200.00         200.00
mean     214.80    10.49          15.56   12.07           0.98
std      169.32     8.26          19.99   17.20           0.04
min        3.00     0.10           0.00    0.30           0.70
25%      109.75     4.07           1.27    3.10           0.98
50%      165.00     8.65           8.45    7.80           1.00
75%      243.00    14.10          19.73   12.92           1.00
max      816.00    40.00          85.00   85.20           1.00

🏆 TOP FOODS BY PROTEIN:
  Viande des grisons: 40.0g protein, 197 kcal
  Viande de grisons: 38.0g protein, 194 kcal
  Bovril Beef and Yeast Extract: 37.0g protein, 206 kcal
  Beef Jerky Original 25g Jack L: 35.2g protein, 252 kcal
  Original Beef Flavor Grass Fed: 31.2g protein, 312 kcal

🌟 TOP FOODS BY QUALITY:
  Salade & Compagnie Manhattan P: Quality 1.00, 186 kcal
  Instant Nudeln - Chicken Flavo: 

## 🎯 Interactive User Profile Experimentation

In [10]:
# !pip install ipywidgets

In [11]:
# Interactive experimentation with different user profiles
try:
    from ipywidgets import interact, IntSlider, FloatSlider, Dropdown
    
    def experiment_user_profile(age=30, weight=75, height=180, activity='MODERATELY_ACTIVE'):
        """Interactive user profile experimentation"""
        test_profile = UserProfile(
            age=age, 
            gender=Gender.MALE, 
            weight=weight, 
            height=height,
            activity_level=ActivityLevel[activity], 
            goals={'weight_goal': Goal.MAINTAIN_WEIGHT}
        )
        
        # Get the summary dictionary
        summary = test_profile.get_summary()
        personal_info = summary['personal_info']
        goals_info = summary['goals']

        print(f"\n👤 Profile Analysis:")
        print(f"BMI: {personal_info['bmi']:.1f} ({personal_info['bmi_category']})")
        print(f"BMR: {goals_info['bmr']:.0f} kcal/day")
        print(f"Target Calories: {goals_info['target_calories']:.0f} kcal/day")
        print(f"Protein: {goals_info['macro_grams'][0]:.1f}g ({goals_info['macro_ratios'][0]:.0%})")
        print(f"Carbs: {goals_info['macro_grams'][1]:.1f}g ({goals_info['macro_ratios'][1]:.0%})")
        print(f"Fat: {goals_info['macro_grams'][2]:.1f}g ({goals_info['macro_ratios'][2]:.0%})")
        
        # Create quick visualization
        macros = goals_info['macro_grams']
        fig = go.Figure(data=[
            go.Pie(labels=['Protein', 'Carbohydrates', 'Fat'],
                   values=[macros[0]*4, macros[1]*4, macros[2]*9],
                   hole=0.3)
        ])
        fig.update_layout(
            title=f"Calorie Distribution - {goals_info['target_calories']:.0f} kcal/day",
            template='plotly_white'
        )
        fig.show()
    
    # Create interactive widgets
    print("🎛️ Interactive User Profile Experimentation:")
    print("Use the sliders below to see how different parameters affect nutritional needs:\n")
    
    interact(
        experiment_user_profile,
        age=IntSlider(min=18, max=80, step=1, value=30, description='Age:'),
        weight=FloatSlider(min=40, max=150, step=5, value=75, description='Weight (kg):'),
        height=IntSlider(min=140, max=220, step=5, value=180, description='Height (cm):'),
        activity=Dropdown(
            options=['SEDENTARY', 'LIGHTLY_ACTIVE', 'MODERATELY_ACTIVE', 'VERY_ACTIVE', 'EXTREMELY_ACTIVE'],
            value='MODERATELY_ACTIVE',
            description='Activity Level:'
        )
    )
    
except ImportError:
    print("💡 Install ipywidgets for interactive controls: pip install ipywidgets")
    print("\nShowing sample profile comparisons instead:")
    
    # Show sample comparisons
    sample_profiles = [
        {'name': 'Young Active', 'age': 25, 'weight': 60, 'height': 165, 'activity': 'VERY_ACTIVE'},
        {'name': 'Middle-aged', 'age': 40, 'weight': 80, 'height': 175, 'activity': 'MODERATELY_ACTIVE'},
        {'name': 'Senior', 'age': 65, 'weight': 70, 'height': 170, 'activity': 'LIGHTLY_ACTIVE'}
    ]
    
    comparison_data = []
    for profile_info in sample_profiles:
        profile = UserProfile(
            age=profile_info['age'], gender='male', 
            weight=profile_info['weight'], height=profile_info['height'],
            activity_level=profile_info['activity'], weight_goal='maintain'
        )
        comparison_data.append({
            'Profile': profile_info['name'],
            'Calories': profile.goals['target_calories'],
            'Protein': profile.goals['macro_grams'][0],
            'Carbs': profile.goals['macro_grams'][1],
            'Fat': profile.goals['macro_grams'][2]
        })
    
    df_comparison = pd.DataFrame(comparison_data)
    
    # Create comparison chart
    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        name='Calories', 
        x=df_comparison['Profile'], 
        y=df_comparison['Calories'],
        marker_color='skyblue',
        text=[f"{cal:.0f}" for cal in df_comparison['Calories']],
        textposition='auto'
    ))
    
    fig.update_layout(
        title='📊 Sample Profile Comparison - Daily Calorie Needs',
        yaxis_title='Calories',
        height=400,
        template='plotly_white'
    )
    
    fig.show()
    
    print("\n📋 Sample Profile Comparison:")
    print(df_comparison.to_string(index=False))

🎛️ Interactive User Profile Experimentation:
Use the sliders below to see how different parameters affect nutritional needs:



interactive(children=(IntSlider(value=30, description='Age:', max=80, min=18), FloatSlider(value=75.0, descrip…

## 🧪 Quick Training Session (Optional)

In [12]:
# Quick training session in notebook
# WARNING: This will train a new agent from scratch!

ENABLE_QUICK_TRAINING = False  # Change to True to enable

if ENABLE_QUICK_TRAINING and RL_AVAILABLE:
    print("🚀 Starting quick training session...")
    print("⚠️  This will create a new agent and train from scratch!")
    
    # Create fresh agent
    quick_agent = PPOAgent(obs_dim, action_dim, config)
    
    # Train for a few episodes with real-time plotting
    training_episodes = 30
    print(f"Training for {training_episodes} episodes...")
    
    quick_history = quick_agent.train(
        environment=env, 
        num_episodes=training_episodes, 
        update_frequency=5
    )
    
    # Plot live results
    rewards = quick_history['episode_rewards']
    episodes = list(range(len(rewards)))
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=episodes, y=rewards,
        mode='lines+markers',
        name='Training Rewards',
        line=dict(color='blue', width=2),
        marker=dict(size=4)
    ))
    
    fig.update_layout(
        title=f'🏃‍♂️ Quick Training Results ({training_episodes} episodes)',
        xaxis_title='Episode',
        yaxis_title='Reward',
        height=400,
        template='plotly_white'
    )
    
    fig.show()
    
    print(f"✅ Quick training completed!")
    print(f"Initial reward: {rewards[0]:.3f}")
    print(f"Final reward: {rewards[-1]:.3f}")
    print(f"Improvement: {((rewards[-1] - rewards[0]) / abs(rewards[0]) * 100 if rewards[0] != 0 else 0):.1f}%")
    
else:
    print("💡 Set ENABLE_QUICK_TRAINING = True to run a quick training session")
    print("   (This will create and train a new agent from scratch)")
    
    if not RL_AVAILABLE:
        print("❌ RL components not available - cannot train")

💡 Set ENABLE_QUICK_TRAINING = True to run a quick training session
   (This will create and train a new agent from scratch)
